In [ ]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_comments(film_id):
    comments_url = f"https://www.allocine.fr/film/fichefilm-{film_id}/critiques/spectateurs/"
    comments_response = requests.get(comments_url)
    comments_soup = BeautifulSoup(comments_response.text, 'html.parser')

    pagination = comments_soup.find('nav', class_='pagination')
    if pagination:
        last_page_elements = pagination.find_all(['a', 'span'], class_='item')
        last_page_element = last_page_elements[-1]
        last_page_link = last_page_element['href'] if last_page_element.name == 'a' else comments_url
        last_page_number = int(last_page_element.get_text())
        pages_urls = [f"{comments_url}?page={page}" for page in range(1, last_page_number + 1)]
    else:
        pages_urls = [comments_url]

    print("URL de la dernière page:", last_page_link)
    print("Numéro de la dernière page:", last_page_number)

    for page_url in pages_urls:
        time.sleep(1)  # Attendre 1 seconde avant de faire la prochaine requête
        comments_response = requests.get(page_url)
        comments_soup = BeautifulSoup(comments_response.text, 'html.parser')

        # Récupérer les notes
        for review in comments_soup.find_all("div", class_="review-card-review-holder"):
            print(review.find("span", class_="stareval-note").get_text())

        # # Récupérer et imprimer les commentaires, sans les spoilers
        # for review in comments_soup.find_all('div', class_='content-txt review-card-content'):
        #     for spoiler in review.find_all('span', class_='spoiler-header'):
        #         spoiler.extract()
        #     text = review.get_text(strip=True, separator='\n')
        #     print(text)

# URL de la liste des meilleurs films
base_url = "https://www.allocine.fr/film/meilleurs/"
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Trouver les liens de films
films_links = soup.find_all('a', class_='meta-title-link')

# Initialiser un compteur pour les films
film_count = 0

# Parcourir les pages de la liste des films et collecter les liens des films
page_number = 1
while film_count < 20:
    for film_link in films_links:
        film_id = film_link['href'].split('cfilm=')[1].split('.')[0]  # Extraire l'ID du film
        film_url = f"https://www.allocine.fr/film/fichefilm_gen_cfilm={film_id}.html"
        print(f"\nTraitement du film : {film_url}")
        scrape_comments(film_id)
        
        film_count += 1
        if film_count >= 20:
            break

    # Passer à la page suivante
    page_number += 1
    next_page_url = f"{base_url}?page={page_number}"
    response = requests.get(next_page_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    films_links = soup.find_all('a', class_='meta-title-link')

print("Scraping terminé.")
